In [1]:
import cv2
import numpy as np
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Robot, Camera, bgr8_to_jpeg
import traitlets
import time
import os 

# =================================================
# 1. INITIALIZATION
# =================================================
robot = Robot()
camera = Camera.instance(width=224, height=224) 

# =================================================
# 2. LOAD CONFIGURATION
# =================================================
def load_color_config():
    y_lower = np.array([20, 50, 50])
    y_upper = np.array([40, 255, 255])
    r_lower = np.array([40, 112, 137])
    r_upper = np.array([179, 255, 255])

    if os.path.exists('yellow_config.npy'):
        try:
            y_conf = np.load('yellow_config.npy')
            y_lower = np.array(y_conf[0:3]); y_upper = np.array(y_conf[3:6])
        except: pass
    if os.path.exists('red_config.npy'):
        try:
            r_conf = np.load('red_config.npy')
            r_lower = np.array(r_conf[0:3]); r_upper = np.array(r_conf[3:6])
        except: pass
    return y_lower, y_upper, r_lower, r_upper

saved_yellow_lower, saved_yellow_upper, saved_red_lower, saved_red_upper = load_color_config()

# =================================================
# 3. STATE VARIABLES
# =================================================
lost_line_count = 0 
last_error = 0       
prev_error = 0 
obstacle_detected = False
ignore_obstacle_time = 0
startup_counter = 0 

# === [NEW FUNCTION] CHECK LINE FROM DISTANCE ===
def check_yellow_far():
    image = camera.value
    # Look from pixel 90 (farther) instead of 150 (closer)
    # This helps the robot see the road earlier when merging back
    image_check = image[90:224, 0:224] 
    hsv_check = cv2.cvtColor(image_check, cv2.COLOR_BGR2HSV)
    mask_check = cv2.inRange(hsv_check, saved_yellow_lower, saved_yellow_upper)
    return cv2.countNonZero(mask_check) > 200

# =================================================
# 4. MAIN LOGIC (UPDATED)
# =================================================
def execute(change):
    global lost_line_count, last_error, prev_error, obstacle_detected, ignore_obstacle_time, startup_counter
    
    if obstacle_detected: return

    image = change['new']
    # Standard ROI Crop
    image_roi = image[90:224, 0:224] 
    hsv = cv2.cvtColor(image_roi, cv2.COLOR_BGR2HSV)
    
    # Soft Start (Startup Delay)
    if startup_counter < 20:
        startup_counter += 1; robot.stop(); return 

    # --- PART 1: OBSTACLE DETECTION ---
    detected_action_now = False 
    approaching_obstacle = False 
    avoid_direction = 'left' 
    
    if time.time() > ignore_obstacle_time:
        mask_red = cv2.inRange(hsv, saved_red_lower, saved_red_upper)
        mask_red = cv2.erode(mask_red, None, iterations=1)
        mask_red = cv2.dilate(mask_red, None, iterations=1)
        red_contours, _ = cv2.findContours(mask_red, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        if len(red_contours) > 0:
            largest_red = max(red_contours, key=cv2.contourArea)
            area_red = cv2.contourArea(largest_red)
            
            if 200 < area_red < 1000: approaching_obstacle = True 
            elif area_red >= 1000:
                M = cv2.moments(largest_red)
                if M['m00'] > 0:
                    cx_red = int(M['m10'] / M['m00'])
                    # Narrow activation zone (only dodge if object really blocks the path)
                    if abs(cx_red - 112) < 55: 
                        detected_action_now = True
                        if cx_red < 112: avoid_direction = 'right'
                        else: avoid_direction = 'left'

    # --- PART 2: EXECUTE AVOIDANCE (MODIFIED: SHORT BLIND + FAR LOOK) ---
    if detected_action_now:
        obstacle_detected = True 
        
        # === STEP 1 & 2: CLEAR THE OBSTACLE ===
        if avoid_direction == 'left':
            robot.set_motors(-0.3, 0.3); time.sleep(0.1) # Turn Out
            robot.set_motors(0.3, 0.3); time.sleep(0.)   # Drive Straight
            # Turn head back towards the line
            robot.set_motors(0.25, -0.25); time.sleep(0.25) 
        else:
            robot.set_motors(0.3, -0.3); time.sleep(0.1)
            robot.set_motors(0.3, 0.3); time.sleep(0.6)
            robot.set_motors(-0.25, 0.25); time.sleep(0.25)
            
        # === STEP 3: SHORT BLIND RUN ===
        # Reduced from 0.8s to 0.3s to avoid overshooting
        robot.set_motors(0.35, 0.35); time.sleep(0.3)
        
        # === STEP 4: ADVANCE AND LOOK FOR LINE (FAR DETECTION) ===
        found_line = False
        start_search = time.time()
        
        # Drive forward while checking (timeout 1.5s)
        while (time.time() - start_search) < 1.5:
            # Use check_yellow_far to see the segment in the distance
            if check_yellow_far():
                found_line = True
                break
            
            # If not found yet, continue driving straight slowly to search
            robot.set_motors(0.35, 0.35) 
            time.sleep(0.05)
            
        # === STEP 5: MERGE BACK / REJOIN LANE ===
        if found_line:
            # When line is detected from afar, curve gently to follow that direction
            if avoid_direction == 'left': 
                robot.set_motors(0.15, 0.45) 
            else: 
                robot.set_motors(0.45, 0.15)
            time.sleep(0.2)
        else:
            robot.stop()
            
        ignore_obstacle_time = time.time() + 3.0 
        lost_line_count = 0 
        last_error = 0
        obstacle_detected = False
        return 

    # --- PART 3: LINE FOLLOWING ---
    mask_yellow = cv2.inRange(hsv, saved_yellow_lower, saved_yellow_upper)
    mask_yellow = cv2.dilate(mask_yellow, None, iterations=1)
    yellow_contours, _ = cv2.findContours(mask_yellow, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(yellow_contours) > 0:
        c = max(yellow_contours, key=cv2.contourArea)
        if cv2.contourArea(c) > 300: 
            lost_line_count = 0 
            M = cv2.moments(c)
            if M['m00'] > 0:
                cx = int(M['m10'] / M['m00'])
                error = cx - 112
                last_error = error
                
                kp = 0.003; kd = 0.005  
                turn = (error * kp) + ((error - prev_error) * kd)
                prev_error = error 
                
                max_spd = 0.20 if approaching_obstacle else 0.45
                speed = max(max_spd - abs(turn), 0.20)
                
                robot.set_motors(min(max(speed + turn, -1.0), 1.0), 
                                 min(max(speed - turn, -1.0), 1.0))
        else:
            lost_line_count += 1
    else:
        lost_line_count += 1
        
    if lost_line_count > 0:
        if lost_line_count < 15: robot.set_motors(0.25, 0.25)
        elif lost_line_count < 40:
            if last_error > 0: robot.set_motors(0.25, -0.2)
            else: robot.set_motors(-0.2, 0.25)
        else: robot.stop()

# =================================================
# 5. ACTIVATION
# =================================================
execute({'new': camera.value}) 
camera.observe(execute, names='value')

🚀 Jetbot VJP PRO: Short Blind & Far Detection Active


In [2]:
import time
camera.unobserve_all()
time.sleep(0.5)
robot.stop()

